In [1]:
import pandas as pd
import plotly.express as px

import calliope

# We increase logging verbosity
calliope.set_log_verbosity("INFO", include_solver_output=False)

model = calliope.read_yaml("model.yaml")

[2025-10-13 08:05:43] INFO     Math init | loading pre-defined math.
[2025-10-13 08:05:43] INFO     Math init | loading math files {'storage_inter_cluster', 'spores', 'operate', 'milp', 'base'}.
[2025-10-13 08:05:43] INFO     Model: preprocessing data
[2025-10-13 08:05:43] INFO     Math build | building applied math with ['base'].
[2025-10-13 08:05:44] INFO     input data `color` not defined in model math; it will not be available in the optimisation problem.
[2025-10-13 08:05:44] INFO     input data `name` not defined in model math; it will not be available in the optimisation problem.
[2025-10-13 08:05:44] WARNING  ModelWarning: Only one timestep defined. Inferring timestep resolution to be 1 hour

[2025-10-13 08:05:44] INFO     input data `color` not defined in model math; it will not be available in the optimisation problem.
[2025-10-13 08:05:44] INFO     input data `name` not defined in model math; it will not be available in the optimisation problem.
[2025-10-13 08:05:44] INFO   

In [2]:
model.inputs

<xarray.Dataset> Size: 2kB
Dimensions:                     (costs: 1, techs: 9, nodes: 4, carriers: 2,
                                 timesteps: 1)
Coordinates:
  * costs                       (costs) object 8B 'monetary'
  * techs                       (techs) object 72B 'N1_to_X2' ... 'supply_gri...
  * carriers                    (carriers) object 16B 'electricity' 'heat'
  * nodes                       (nodes) object 32B 'N1' 'X1' 'X2' 'X3'
  * timesteps                   (timesteps) datetime64[ns] 8B 2050-01-01
Data variables: (12/23)
    cost_interest_rate          (costs) float64 8B 0.1
    bigM                        float64 8B 1e+06
    objective_cost_weights      (costs) float64 8B 1.0
    base_tech                   (techs) object 72B 'transmission' ... 'supply'
    carrier_in                  (nodes, techs, carriers) bool 72B False ... F...
    color                       (techs) object 72B '#823739' ... '#C5ABE3'
    ...                          ...
    longitude                   (nodes) float64 32B 4.359 4.358 4.361 4.362
    sink_use_equals             (timesteps, techs, nodes) float64 288B nan .....
    definition_matrix           (nodes, techs, carriers) bool 72B False ... F...
    distance                    (techs) float64 72B 0.1734 0.2448 ... nan nan
    timestep_resolution         (timesteps) float64 8B 1.0
    timestep_weights            (timesteps) float64 8B 1.0

In [3]:
model.inputs.flow_cap_max.to_series().dropna()

techs
N1_to_X2             2000.0
N1_to_X3             2000.0
X1_to_N1             2000.0
X1_to_X2             2000.0
X1_to_X3             2000.0
supply_geothermal    2000.0
supply_grid_power    2000.0
Name: flow_cap_max, dtype: float64

In [4]:
model.inputs.sink_use_equals.sum(
    "timesteps", min_count=1, skipna=True
).to_series().dropna()

techs               nodes
demand_electricity  X1         0.637704
                    X2       101.809815
                    X3        20.592169
demand_heat         X1         4.348486
                    X2       128.210300
                    X3        25.173941
Name: sink_use_equals, dtype: float64

In [5]:
model.build()
model.solve()

[2025-10-13 08:05:55] INFO     Model: backend build starting
[2025-10-13 08:05:55] INFO     Optimisation Model | parameters/lookups | Generated.
[2025-10-13 08:05:55] INFO     Optimisation Model | variables | Generated.
[2025-10-13 08:05:56] INFO     Optimisation Model | global_expressions | Generated.
[2025-10-13 08:05:57] INFO     Optimisation Model | constraints | Generated.
[2025-10-13 08:05:57] INFO     Optimisation Model | piecewise_constraints | Generated.
[2025-10-13 08:05:57] INFO     Optimisation Model | objectives | Generated.
[2025-10-13 08:05:57] INFO     Model: backend build complete
[2025-10-13 08:05:57] INFO     Optimisation model | starting model in base mode.
[2025-10-13 08:05:58] INFO     Backend: solver finished running. Time since start of solving optimisation problem: 0:00:00.418106
[2025-10-13 08:05:58] INFO     Postprocessing: applied zero threshold 1e-10 to model results.
[2025-10-13 08:05:58] INFO     Postprocessing: ended. Time since start of solving optimisa

In [6]:
model.results

<xarray.Dataset> Size: 6kB
Dimensions:                     (nodes: 4, techs: 9, carriers: 2, timesteps: 1,
                                 costs: 1)
Coordinates:
  * techs                       (techs) object 72B 'N1_to_X2' ... 'supply_gri...
  * nodes                       (nodes) object 32B 'N1' 'X1' 'X2' 'X3'
  * carriers                    (carriers) object 16B 'electricity' 'heat'
  * timesteps                   (timesteps) datetime64[ns] 8B 2050-01-01
  * costs                       (costs) object 8B 'monetary'
Data variables: (12/21)
    flow_cap                    (nodes, techs, carriers) float64 576B nan ......
    link_flow_cap               (techs) float64 72B 0.0 0.0 0.0 ... nan nan nan
    flow_out                    (nodes, techs, carriers, timesteps) float64 576B ...
    flow_in                     (nodes, techs, carriers, timesteps) float64 576B ...
    source_use                  (nodes, techs, timesteps) float64 288B nan .....
    source_cap                  (nodes, techs) float64 288B nan nan ... nan
    ...                          ...
    min_cost_optimisation       float64 8B 12.71
    capacity_factor             (nodes, techs, carriers, timesteps) float64 576B ...
    systemwide_capacity_factor  (techs, carriers) float64 144B 0.0 0.0 ... 0.0
    systemwide_levelised_cost   (techs, costs, carriers) float64 144B nan ......
    total_levelised_cost        (costs, carriers) float64 16B 0.1013 0.0806
    unmet_sum                   (nodes, carriers, timesteps) float64 64B nan ...

In [7]:
df_heat = (
    model.results.flow_out.sel(carriers="heat")
    .sum("nodes", min_count=1, skipna=True)
    .to_series()
    .dropna()
    .unstack("techs")
)

df_heat.head()

techs,N1_to_X2,N1_to_X3,X1_to_N1,supply_geothermal
timesteps,,,,
2050-01-01,0.0,0.0,0.0,157.732727


In [8]:
costs = model.results.cost.to_series().dropna()
costs.head()

nodes  techs     costs   
N1     N1_to_X2  monetary    0.000000
       N1_to_X3  monetary    0.000000
       X1_to_N1  monetary    0.000000
X1     X1_to_N1  monetary    0.000000
       X1_to_X2  monetary    0.000002
Name: cost, dtype: float64

In [9]:
lcoes = (
    model.results.systemwide_levelised_cost.sel(carriers="electricity")
    .to_series()
    .dropna()
)
lcoes.head()

techs              costs   
X1_to_X2           monetary    4.514251e-08
X1_to_X3           monetary    7.388033e-08
supply_grid_power  monetary    1.012576e-01
Name: systemwide_levelised_cost, dtype: float64

In [10]:
# We set the color mapping to use in all our plots by extracting the colors defined in the technology definitions of our model.
colors = model.inputs.color.to_series().to_dict()

df_electricity = (
    (model.results.flow_out.fillna(0) - model.results.flow_in.fillna(0))
    .sel(carriers="electricity")
    .sum("nodes")
    .to_series()
    .where(lambda x: x != 0)
    .dropna()
    .to_frame("Flow in/out (kWh)")
    .reset_index()
)
df_electricity_demand = df_electricity[df_electricity.techs == "demand_electricity"]
df_electricity_other = df_electricity[df_electricity.techs != "demand_electricity"]

print(df_electricity.head())

fig1 = px.bar(
    df_electricity_other,
    x="timesteps",
    y="Flow in/out (kWh)",
    color="techs",
    color_discrete_map=colors,
)
fig1.add_scatter(
    x=df_electricity_demand.timesteps,
    y=-1 * df_electricity_demand["Flow in/out (kWh)"],
    marker_color="black",
    name="demand",
)

                techs  timesteps  Flow in/out (kWh)
0            X1_to_X2 2050-01-01          -2.077751
1            X1_to_X3 2050-01-01          -0.420248
2  demand_electricity 2050-01-01        -123.039687
3   supply_grid_power 2050-01-01         125.537687


In [49]:
carriers = ["heat", "electricity"]
df_flows = (
    (model.results.flow_out.fillna(0) - model.results.flow_in.fillna(0))
    .sel(carriers=carriers)
    .to_series()
    .where(lambda x: x != 0)
    .dropna()
    .to_frame("Flow in/out (kWh)")
    .reset_index()
)
df_demand = df_flows[df_flows.techs.str.contains("demand")]
df_flows_other = df_flows[~df_flows.techs.str.contains("demand")]

print(df_flows.head())

node_order = df_flows_other.nodes.unique()

fig = px.bar(
    df_flows_other,
    x="timesteps",
    y="Flow in/out (kWh)",
    facet_row="nodes",
    facet_col="carriers",
    color="techs",
    category_orders={"nodes": node_order, "carriers": carriers},
    height=1000,
    color_discrete_map=colors,
)

showlegend = True
# we reverse the node order (`[::-1]`) because the rows are numbered from bottom to top.
for row, node in enumerate(node_order[::-1]):
    for col, carrier in enumerate(carriers):
        demand_ = df_demand.loc[
            (df_demand.nodes == node) & (df_demand.techs == f"demand_{carrier}"),
            "Flow in/out (kWh)",
        ]
        if not demand_.empty:
            fig.add_scatter(
                x=model.results.timesteps.values,
                y=-1 * demand_,
                row=row + 1,
                col=col + 1,
                marker_color="black",
                name="Demand",
                legendgroup="demand",
                showlegend=showlegend,
            )
            showlegend = False
fig.update_yaxes(matches=None)
fig.show()

  nodes     techs     carriers  timesteps  Flow in/out (kWh)
0    N1  N1_to_X2         heat 2050-01-01         -89.054191
1    N1  X1_to_N1         heat 2050-01-01          89.054191
2    X1  X1_to_N1         heat 2050-01-01         -96.081664
3    X1  X1_to_X2  electricity 2050-01-01        -103.887566
4    X1  X1_to_X3  electricity 2050-01-01         -21.012417


In [50]:
df_capacity = (
    model.results.flow_cap.where(
        ~model.inputs.base_tech.str.contains("demand|transmission")
    )
    .to_series()
    .where(lambda x: x != 0)
    .dropna()
    .to_frame("Flow capacity (kW)")
    .reset_index()
)

print(df_capacity.head())

fig = px.bar(
    df_capacity,
    x="nodes",
    y="Flow capacity (kW)",
    color="techs",
    facet_col="carriers",
    color_discrete_map=colors,
)
fig.show()

  nodes       techs     carriers  Flow capacity (kW)
0    X1         chp  electricity          125.537687
1    X1         chp          gas          309.969597
2    X1         chp         heat          100.430149
3    X1  supply_gas          gas          309.969597
4    X2      boiler          gas           53.728928


In [66]:
df_coords = model.inputs[["latitude", "longitude"]].to_dataframe().reset_index()
df_capacity = (
    model.results.flow_cap.where(model.inputs.base_tech == "transmission")
    .to_series()
    .where(lambda x: x != 0)
    .dropna()
    .to_frame("Flow capacity (kW)")
    .reset_index()
)
df_capacity_coords = pd.merge(df_coords, df_capacity, left_on="nodes", right_on="nodes")

fig = px.line_map(
    df_capacity_coords,
    lat="latitude",
    lon="longitude",
    color="carriers",
    hover_name="nodes",
    hover_data="Flow capacity (kW)",
    zoom=3,
    height=1200,
)
fig.update_layout(
    map_style="open-street-map",
    map_zoom=16.2,
    map_center_lat=df_coords.latitude.mean(),
    map_center_lon=df_coords.longitude.mean(),
    margin={"r": 0, "t": 0, "l": 0, "b": 0},
    hoverdistance=50,
)